In [164]:
import MySQLdb
import pyvo
import numpy as np
import re
from astropy.table import QTable, hstack
from astropy.coordinates import Angle, SkyCoord
from astropy import units as u
from astropy.io import ascii
import os.path
import requests
from bs4 import BeautifulSoup
import pandas as pd
from astroquery.ned import Ned

#### AMIGA database keys (contact an admin for a login)

In [2]:
host = xxx
user = xxx
passwd = xxx

### Utility functions

In [204]:
def retrieve_amiga(user, passwd, wholeamiga=False):
    '''
    Function to retrieve the AMIGA sample from the database.
    Use the wholeamiga key to retrieve either the full CIG sample of 1051 galaxies (=True),
    or the final sample of Verley+07 (=False)
    '''
    db = MySQLdb.connect(db='CIG_RELEASE_2012', host=host, user=user, passwd=passwd)

    query_sub = "SELECT c.cig, c.ASC_D_J2000, c.DEC_D_J2000, r.BTC, v.VELOC_HC, i.Q, i.nK \
    FROM CIG_RELEASE_2012.COORDINATES c \
    LEFT JOIN CIG_RELEASE_2012.RESULTS_OPT r \
    ON r.cig = c.cig \
    LEFT JOIN CIG_RELEASE_2012.VELOCITIES v \
    ON v.cig = c.cig \
    LEFT JOIN PAPERS_ISOLATION_VERLEY07a.TABLE4 i \
    ON i.cig = c.cig \
    WHERE r.BTC <= 15.3 AND r.BTC  > 11 AND v.VELOC_HC >= 1500 \
    AND ((i.Q < -2 AND i.nK < 2.4 AND i.Q IS NOT NULL) OR (i.nK IS NULL AND i.Q < -2)) \
    AND (r.BTC > 11 and r.BTC < 15.3)\
    ORDER BY c.cig"
    cols = ('CIG', 'RA', 'DEC', 'Bmag', 'VEL', 'Q', 'nK')

    query_whole = "SELECT c.cig, c.ASC_D_J2000, c.DEC_D_J2000, r.BTC, v.VELOC_HC, i.Q, i.nK \
    FROM CIG_RELEASE_2012.COORDINATES c \
    LEFT JOIN CIG_RELEASE_2012.RESULTS_OPT r \
    ON r.cig = c.cig \
    LEFT JOIN CIG_RELEASE_2012.VELOCITIES v \
    ON v.cig = c.cig \
    LEFT JOIN PAPERS_ISOLATION_VERLEY07a.TABLE4 i \
    ON i.cig = c.cig \
    ORDER BY c.cig"
    cols = ('CIG', 'RA', 'DEC', 'Bmag', 'VEL', 'Q', 'nK')
    
    if wholeamiga == True:
        query = query_whole
    else:
        query = query_sub
        
    cursor = db.cursor()
    cursor.execute(query)
    cursor.scroll(0, 'absolute')
    nrows = cursor.rowcount
    ncols = len(cursor.fetchone())
    data = np.zeros([nrows,ncols])
    cursor.scroll(0, 'absolute')
    for i in range(nrows):
        row = cursor.fetchone()
        for j in range(ncols):
            try:
                data[i,j] = float(row[j])
            except TypeError:
                data[i,j] = row[j]
    table = QTable(data, names=cols)
    table['CIG'].info.format = '%d'
    return table

############################################

def isinVerley(cigno):
    '''Function to check if a CIG is included in the final AMIGA sample of Verley+07'''
    amg_tab = retrieve_amiga(user, passwd, wholeamiga=False)
    row = None
    for i in range(len(amg_tab['CIG'])):
        if cigno == amg_tab['CIG'][i]:
            row = amg_tab[i]
    if row:
        print('CIG %d: B = %.2f mag & V = %.1f km/s -- RA, Dec = %.6f, %.6f' %(row[0],row[3],row[4], row[1], row[2]))
 
    else:
        print('CIG %d is not in the Verley+07 sample' %cigno)

############################################

def searchamiga(name=None, coords=None, radius=1.0, wholeamiga=True):
    '''
    Function to search in AMIGA for an individual galaxy.
    - galname: NED name of galaxy
    - radius: search radius in arcmin
    '''
    if name:
        galquery = Ned.query_object(name)
        galra, galdec = galquery['RA'][0], galquery['DEC'][0]
        coords_obj = SkyCoord(galra*u.deg, galdec*u.deg, frame='icrs')
    elif coords:
        try:
            galra = np.float(re.split(',|;| ', coords)[0])
            galdec = np.float(re.split(',|;| ', coords)[-1])
            coords_obj = SkyCoord(galra*u.deg, galdec*u.deg, frame='icrs')
        except:
            coords = coords.replace(',','')
            coords = coords.replace(';','')
            coords_obj = SkyCoord(coords, unit=(u.hourangle, u.deg))
    radius = float(radius)*u.arcmin
    amg_tab = retrieve_amiga(user, passwd, wholeamiga)
    ra_a, dec_a = amg_tab['RA'], amg_tab['DEC']
    coords_amg = SkyCoord(ra=ra_a*u.degree, dec=dec_a*u.degree, frame='icrs')
    idx, d2d, d3d = coords_obj.match_to_catalog_sky(coords_amg)
    sep = d2d < radius
    if sep:
        print('Matching AMIGA object: CIG %d with %.3f arcsec' %(amg_tab[idx]['CIG'],d2d.arcsec))
    else:
        print("There's no match in AMIGA within ", radius)
        
############################################

def retrieve_viz(url, table, cols):
    '''Function to retrieve Vizier table at specific url and columns.'''
    tap = pyvo.dal.TAPService(url) 
    hardlim = tap.hardlimit
    result = tap.search('SELECT %s FROM "%s"' %(cols, table), maxrec=int(0.1*hardlim))
    return result.to_table()

############################################

def xmatch_amiga_viz(radius_arcmin, wholeamiga=True):
    '''
    Cross-match between the AMIGA sample and the Vizier table of CALIFA sample.
    radius_arcmin: search radius in arcmin
    '''
    
    ### Retrieving the AMIGA sample
    amg_tab = retrieve_amiga(user, passwd, wholeamiga)

    ### Retrieving the Vizier table
    viz_url = "http://TAPVizieR.u-strasbg.fr/TAPVizieR/tap"
    viz_table = 'J/A+A/632/A59/tableb1'
    viz_cols = 'ID AS CALIFA_ID, _RA, _DE, SimbadName AS Name, SFR, rMAG, lRe, l2Re'
    viz_tab = retrieve_viz(viz_url, viz_table, viz_cols)

    ### Matching AMIGA and Vizier tables
    ra_a, dec_a = amg_tab['RA'], amg_tab['DEC']
    ra_v, dec_v = viz_tab['_RA'], viz_tab['_DE']
    coords_amg = SkyCoord(ra=ra_a*u.degree, dec=dec_a*u.degree, frame='icrs')
    coords_viz = SkyCoord(ra=ra_v, dec=dec_v, frame='icrs')
    idx_amg, idx_viz, d2d, d3d = coords_viz.search_around_sky(coords_amg, float(radius_arcmin)*u.arcmin)

    match_amg, match_viz = amg_tab[idx_amg], viz_tab[idx_viz]

    match_tab = hstack([match_amg, match_viz])
    match_tab.add_column(d2d, name='separation')
    match_tab['separation'].info.format = '5.3e'
    
    print('---%d matches were found' %len(match_amg))
    outfile='amiga-califa-lre-%sarcmin.csv' %radius_arcmin
    ascii.write(match_tab, outfile, format='csv', overwrite=True)
    if os.path.isfile(outfile):
        print('---output file saved as %s' %outfile)
    else:
        print('---the file could not be saved')

############################################

def xmatch_sample(sample, radius_arcmin=1, wholeamiga=True, ascii_table=None, id_c='Name', ra_c='RA', dec_c='DEC', refcode=None, output=None):
    '''
    Cross-match between the AMIGA sample and various HI samples.
    Optional:
    - radius_arcmin: cross-match radius in arcmin, default = 20arcmin
    - output: name of output file where to save results of cross-match. Default = do not save
    * If local table (has to be an ascii file):
    - sample = user
    - ascii_table: name of local table. Must be ASCII table, and have at least 3 columns: ID, RA and Dec.
    - id_c: column name for object name; default = 'Name' -- CANNOT be 'CIG'!
    - ra_c: column name for RA; default = 'RA'
    - dec_c: column name for Dec; default = 'DEC'
    * If table on NED:
    - sample = 'NED' or 'ned' (case insensitive)
    - refcode: reference code for table, e.g: 2017MNRAS.469.2387P
    * If table on SIMBAD:
    - sample = 'SIMBAD' or 'simbad' (case insensitive)
    - refcode: reference code for table, e.g: 2017MNRAS.469.2387P
    '''
            
    amg_tab = retrieve_amiga(user, passwd, wholeamiga)
    ra_a, dec_a = amg_tab['RA'], amg_tab['DEC']
    coords_amg = SkyCoord(ra=ra_a*u.degree, dec=dec_a*u.degree, frame='icrs')
    
    ### SPARC sample from Lelli+ 2016 (Table c), locally available and downloaded from http://astroweb.cwru.edu/SPARC/SPARC_Lelli2016c.mrt
    if sample == 'SPARC':
        f = open('SPARC_Lelli2016c.mrt', 'r')
        lines = f.readlines()
        f.close()
        keys = []
        for line in lines[9:28]:
            keys.append(line[28:38].strip())
        tb = pd.read_csv('SPARC_Lelli2016c.mrt', engine='python', skiprows=98, names=keys, sep='\s{1,}')
        no_id, wra, wdec = [], [], []
        for i, gal in enumerate(tb['Galaxy']):
            try:
                w = Ned.query_object(gal)
                wra.append(w['RA'][0])
                wdec.append(w['DEC'][0])
            except:
                no_id.append(i)
                continue
        ra_s, dec_s = np.array(wra), np.array(wdec)
        tb = tb.drop(no_id)
        tab_s = QTable.from_pandas(tb)
        idcolumn = 'Galaxy'
        
    ### Ponomareva+ 2017 (https://ui.adsabs.harvard.edu/abs/2017MNRAS.469.2387P/abstract) sample from NED
    elif sample == 'Ponomareva':
        tab_s = Ned.query_refcode('2017MNRAS.469.2387P')
        ra_s = np.array(tab_s['RA'])
        dec_s = np.array(tab_s['DEC'])
        idcolumn = 'Object Name'

    ### The WHISP sample, locally available and compiled from http://www.astro.rug.nl/~whisp
    elif sample == 'WHISP':
        f = open('whisp_catalogue.txt', 'r')
        lines = f.readlines()
        objlist = []
        for line in lines:
            if line.strip().startswith('UGC'):
                ugcno = line.split()[1]
                try:
                    ugcno = int(ugcno[:5])
                except ValueError:
                    ugcno = int(ugcno[:4])
                objlist.append(line.split()[0] + ' ' + str(ugcno))
        wra, wdec = [], []
        for gal in objlist:
            w = Ned.query_object(gal)
            wra.append(w['RA'][0])
            wdec.append(w['DEC'][0])
        ra_s, dec_s = np.array(wra), np.array(wdec)
        tb = pd.DataFrame(np.array([objlist, wra, wdec]).T, columns=['Galaxy', 'RA', 'DEC'])
        tab_s = QTable.from_pandas(tb)
        idcolumn = 'Galaxy'
        
    ### The THINGS sample from NED
    elif sample == 'THINGS':
        tab_s = Ned.query_refcode('2008AJ....136.2648D')
        ra_s = np.array(tab_s['RA'])
        dec_s = np.array(tab_s['DEC'])
        idcolumn = 'Object Name'

    ### The HALOGAS sample from NED
    elif sample == 'HALOGAS':
        tab_s = Ned.query_refcode('2011A&A...526A.118H')
        ra_s = np.array(tab_s['RA'])
        dec_s = np.array(tab_s['DEC'])
        idcolumn = 'Object Name'

    elif sample.lower() == 'user':
        tab_s = ascii.read(ascii_table)
        ra_s = np.array(tab_s[ra_c])
        dec_s = np.array(tab_s[dec_c])
        idcolumn = id_c
    
    elif sample.lower() == 'ned':
        tab_s = Ned.query_refcode(refcode)
        ra_s = np.array(tab_s['RA'])
        dec_s = np.array(tab_s['DEC'])
        idcolumn = 'Object Name'

    elif sample.lower() == 'simbad':
        tab_s = Simbad.query_bibobj(refcode)
        idcolumn = 'MAIN_ID'
        sb = np.char.add(np.char.add(tab_s['RA'], ' '), tab_s['DEC'])
        sb_c = SkyCoord(c0, unit=(u.hourangle, u.deg))
        ra_s, dec_s = sb_c.ra.degree, sb_c.dec.degree
        
    coords_s = SkyCoord(ra=ra_s*u.degree, dec=dec_s*u.degree, frame='icrs')
    idx_amg, idx_s, d2d, d3d = coords_s.search_around_sky(coords_amg, float(radius_arcmin)*u.arcmin)

    match_amg, match_s = amg_tab[idx_amg], tab_s[idx_s]
    match_amg.add_column([Ned.query_object('CIG %d' %x)['Object Name'][0] for x in match_amg['CIG']], name='XName', index=1)

    match_tab = hstack([match_amg, match_s[idcolumn]])
    match_tab.add_column(d2d, name='separation')
    match_tab['separation'].info.format = '5.3e'

    if not isinstance(output, type(None)):
        fout = open(output, 'w')
        if sample == 'WHISP':
            fout.write('CIG-ID'+3*' '+'UGC-ID'+3*' '+'NED-ID'+12*' '+'VEL'+7*' '+'Qkar'+6*' '+'nk\n')
            for cig, gal, sname, vel, qkar, nk in zip(match_tab['CIG'], match_tab['XName'], match_tab[idcolumn], match_tab['VEL'], match_tab['Q'], match_tab['nK']):
                fout.write('%-8d%-10s%-18s%-10s%-10s%-10s\n' %(cig,sname[3:],gal,vel,qkar,nk))
        else:
            fout.write('CIG-ID'+2*' '+'NED-ID'+12*' '+'VEL'+7*' '+'Qkar'+6*' '+'nk\n')
            for cig, gal, vel, qkar, nk in zip(match_tab['CIG'], match_tab['XName'], match_tab['VEL'], match_tab['Q'], match_tab['nK']):
                fout.write('%-8d%-18s%-10s%-10s%-10s\n' %(cig,gal,vel,qkar,nk))
        fout.close()
        print('--- Number of matches: %d\n--- Output saved to %s' %(len(match_amg),output))

    else:
        print('--- Number of matches: ', len(match_amg))
        print('---------------------------')
        if sample == 'WHISP':
            print('CIG-ID'+3*' '+'UGC-ID'+3*' '+'NED-ID'+12*' '+'VEL'+7*' '+'Qkar'+6*' '+'nk')
            for cig, gal, sname, vel, qkar, nk in zip(match_tab['CIG'], match_tab['XName'], match_tab[idcolumn], match_tab['VEL'], match_tab['Q'], match_tab['nK']):
                print('%-8d%-10s%-18s%-10s%-10s%-10s' %(cig,sname[3:],gal,vel,qkar,nk))
        else:
            print('CIG-ID'+2*' '+'NED-ID'+12*' '+'VEL'+7*' '+'Qkar'+6*' '+'nk')
            for cig, gal, vel, qkar, nk in zip(match_tab['CIG'], match_tab['XName'], match_tab['VEL'], match_tab['Q'], match_tab['nK']):
                print('%-8d%-18s%-10s%-10s%-10s' %(cig,gal,vel,qkar,nk))

## Cross-matching with catalogues

#### Example 1: x-macth with a user-defined, local table in a 2 arcmin radius
##### Accepts ASCII tables: CSV, txt, etc. Must have at least 3 columns: ID, RA & Dec

In [207]:
xmatch_sample('user', 2, ascii_table='table.csv', id_c='Name', ra_c='RA', dec_c='DEC')

--- Number of matches:  17
---------------------------
CIG-ID  NED-ID            VEL       Qkar      nk
1       UGC 00005         7299.0    -1.704    1.814     
33      NGC 0237          4175.0    -1.758    1.634     
81      NGC 0781          3483.0    -2.698    1.457     
128     NGC 1349          6596.0    -3.609    0.587     
145     NGC 1542          3714.0    -3.253    0.736     
196     UGC 03899         3884.0    -0.901    1.044     
199     UGC 03944         3895.0    -2.528    1.703     
293     UGC 04722         1794.0    -3.156    1.244     
340     IC 2487           4339.0    -2.205    nan       
575     NGC 5016          2612.0    -2.802    1.322     
631     NGC 5633          2334.0    -1.433    2.156     
805     IC 1256           4730.0    -2.79     1.695     
828     UGC 10972         4652.0    -2.503    nan       
897     NGC 7025          4968.0    -2.667    1.0       
1020    NGC 7683          3726.0    -0.589    1.627     
1047    UGC 12857         2459.0    -1.21

#### Example 2: x-match with a NED table (Jarrett+ 2019) in a 1 arcmin radius and saving output to file

In [208]:
xmatch_sample('ned', 1.0, refcode='2019ApJS..245...25J')

--- Number of matches:  8
---------------------------
CIG-ID  NED-ID            VEL       Qkar      nk
197     NGC 2403          131.0     nan       nan       
324     NGC 2841          638.0     nan       nan       
347     NGC 2903          556.0     nan       nan       
388     Sextans B         301.0     nan       nan       
461     NGC 3521          805.0     nan       nan       
523     NGC 4236          -5.0      nan       nan       
559     MESSIER 064       408.0     nan       nan       
610     MESSIER 101       241.0     nan       nan       


#### Example 3: x-match with the WHISP sample within 3 arcmin and saving output to file

In [209]:
xmatch_sample('WHISP', 3.0, output='output.txt')

--- Number of matches: 36
--- Output saved to output.txt


## Searching for single objects in AMIGA

#### Example 1: based on NED name, within 1 arcmin

In [210]:
searchamiga(name='UGC 6780', )

Matching AMIGA object: CIG 502 with 1.437 arcsec


#### Example 2: based on coordinates, within 1.5 arcmin

In [211]:
### coordinates in degrees:
searchamiga(coords='10.866007, -0.124929', radius=1.5)

### coordinates in HMS, DMS:
searchamiga(coords='00h43m27.842s, -00d07m29.74s', radius=1.5)

Matching AMIGA object: CIG 33 with 3.025 arcsec
Matching AMIGA object: CIG 33 with 3.021 arcsec
